In [1]:
from scraper import number_companies, valuetoday_usa_scraper, df_cleaning, save_csv

In [2]:
number_companies()

5083

In [4]:
dataframe = valuetoday_usa_scraper(150)
final_df = df_cleaning(dataframe)

METLIFE


ValueError: arrays must all be same length

In [4]:
save_csv(final_df, "value_today")

In [11]:
final_df

,Company Name,Company Business,Market Value,Annual Revenue,Operating Income,Net Income,Assets,Liabilities
0,APPLE,Technology,2256000000000,274510000000,66280000000,274510000000,317344000000,245062000000
1,MICROSOFT CORPORATION,Technology,1682000000000,143000000000,52950000000,143000000000,301311000000,183007000000
2,ALPHABET,Technology,1185000000000,182520000000,41220000000,182520000000,299243000000,86320000000
3,AMAZON.COM,eCommerce,1634000000000,386060000000,22890000000,386060000000,110908000000,184586000000
4,FACEBOOK,Communication Services,778040000000,85960000000,32670000000,85960000000,139691000000,29244000000
5,BERKSHIRE HATHAWAY,Financial Services,543679999999,327220000000,0,327220000000,829946000000,410730000000
6,J P MORGAN CHASE & CO,Financial Services,387330000000,122900000000,0,122900000000,3386000000000,3106000000000
7,VISA,Financial Services,482170000000,21840000000,14090000000,21840000000,77884000000,42360000000
8,JOHNSON & JOHNSON,Healthcare,414310000000,82580000000,19730000000,82580000000,158380000000,95402000000
9,WALMART,Retail,407840000000,523960000000,22540000000,523960000000,237382000000,162072000000


In [4]:
import requests
import re
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [28]:
ua = UserAgent()
header = {'User-Agent':str(ua.chrome)}
url = "https://www.value.today/headquarters/united-states-america-usa?page=6"

source = requests.get(url, headers=header)
soup = BeautifulSoup(source.content, "html.parser")

# Create a list of divs with all required information.
info_blocks = soup.find_all("div", class_="node node--type-listed-companies node--view-mode-teaser ds-2col-stacked clearfix")

collected_names = []
collected_businesses = []
collected_values = []
collected_values_metrics = []
collected_revenues = []
collected_revenues_metrics = []
collected_op_incomes = []
collected_op_incomes_metrics = []
collected_net_incomes = []
collected_net_incomes_metrics = []
collected_assets = []
collected_assets_metrics = []
collected_liabilities = []
collected_liabilities_metrics = []

for item in info_blocks:

    def collect_values(item, collected_list):
        if len(item) == 0:
            collected_list.append(None)
        else:
            string = item[0].text.strip() 
            value = re.findall("\d+[.,]\d+|\b(?<!-)\d+\b", string)
            collected_list.append(value[0])


    def collect_metrics(item, collected_list):
        if len(item) == 0:
            collected_list.append(None)
        else:
            string = item[0].text.strip()
            if "Billion" in string:
                collected_list.append("Billion")
            if "Million" in string:
                collected_list.append("Million")

    market_value_item = item.select("div.field--name-field-market-value-jan012021 > div.field--item")
    if len(market_value_item) == 0:
        continue
    else:
        market_value = market_value_item[0]["content"]
        collected_values.append(market_value)

    collect_metrics(market_value_item, collected_values_metrics)
    
    name = item.find("h2").text.strip()
    collected_names.append(name)

    business = item.select("div.field--name-field-company-category-primary > div > div.field--item > a")[0].text.strip()
    collected_businesses.append(business)

    revenue_item = item.select("div.field--name-field-annual-revenue > div.field--item")
    collect_values(revenue_item, collected_revenues)
    collect_metrics(revenue_item, collected_revenues_metrics)

    op_income_item = item.select("div.field--name-field-annual-operating-income > div.field--item")
    collect_values(op_income_item, collected_op_incomes)
    collect_metrics(op_income_item, collected_op_incomes_metrics)

    net_income_item = item.select("div.field--name-field-annual-revenue > div.field--item")
    collect_values(net_income_item, collected_net_incomes)
    collect_metrics(net_income_item, collected_net_incomes_metrics)

    assets_item = item.select("div.field--name-field-total-assets > div.field--item")
    collect_values(assets_item, collected_assets)
    collect_metrics(assets_item, collected_assets_metrics)

    liabilities_item = item.select("div.field--name-field-total-liabilities- > div.field--item")
    collect_values(liabilities_item, collected_liabilities)
    collect_metrics(liabilities_item, collected_liabilities_metrics)

    df = pd.DataFrame({"Company Name": collected_names, "Company Business": collected_businesses, "Market Value": collected_values, 
                        "Value Metric": collected_values_metrics, "Annual Revenue": collected_revenues, 
                        "Revenue Metric": collected_revenues_metrics, "Operating Income": collected_op_incomes, 
                        "Op Income Metric": collected_op_incomes_metrics, "Net Income": collected_net_incomes, 
                        "Net Income Metric": collected_net_incomes_metrics, "Assets": collected_assets, 
                        "Assets Metric": collected_assets_metrics, "Liabilities": collected_liabilities, 
                        "Liabilities Metric": collected_liabilities_metrics})

In [29]:
df

,Company Name,Company Business,Market Value,Value Metric,Annual Revenue,Revenue Metric,Operating Income,Op Income Metric,Net Income,Net Income Metric,Assets,Assets Metric,Liabilities,Liabilities Metric
0,QUALCOMM,Technology,172.3,Billion,23.53,Billion,6.25,Billion,23.53,Billion,"32,328",Million,"29,022",Million
1,AMERICAN TOWER CORPORATION (REIT),Real Estate,99.708,Billion,7.58,Billion,None,None,7.58,Billion,"41,515",Million,"37,353",Million
2,THE GOLDMAN SACHS GROUP,Financial Services,90.734,Billion,"33,294",Million,None,None,"33,294",Million,None,None,None,None
3,UNITED PARCEL SERVICE (UPS),Industries,145.56,Billion,"84,628",Million,"7,814",Million,"84,628",Million,"62,057",Million,"57,694",Million
4,CVS HEALTH CORPORATION,Healthcare,89.399,Billion,"156,776",Million,None,None,"156,776",Million,"235,495",Million,"167,473",Million
5,U.S. BANCORP,Financial Services,70.185,Billion,"22,883",Million,None,None,"22,883",Million,"546,652",Million,"494,802",Million
6,LOWE'S COMPANIES,Consumer Cyclical,117.61,Billion,89.59,Billion,9.64,Billion,89.59,Billion,46.73,Billion,45.29,Billion


In [27]:
df[df["Company Name"] == "YOUTUBE"]

,Company Name,Company Business,Market Value,Value Metric,Annual Revenue,Revenue Metric,Operating Income,Op Income Metric,Net Income,Net Income Metric,Assets,Assets Metric,Liabilities,Liabilities Metric
3,YOUTUBE,Social Media,None,None,None,None,None,None,None,None,None,None,None,None


In [19]:
info_blocks[3].select("div.field--name-field-market-value-jan012021 > div.field--item")

[]

In [6]:
data = pd.read_csv("value_today.csv")

In [7]:
data

,Company Name,Company Business,Market Value,Annual Revenue,Operating Income,Net Income,Assets,Liabilities
0,APPLE,Technology,2256000000000,274510000000,66280000000,274510000000,317344000000,245062000000
1,MICROSOFT CORPORATION,Technology,1682000000000,143000000000,52950000000,143000000000,301311000000,183007000000
2,ALPHABET,Technology,1185000000000,182520000000,41220000000,182520000000,299243000000,86320000000
3,AMAZON.COM,eCommerce,1634000000000,386060000000,22890000000,386060000000,110908000000,184586000000
4,FACEBOOK,Communication Services,778040000000,85960000000,32670000000,85960000000,139691000000,29244000000
5,BERKSHIRE HATHAWAY,Financial Services,543679999999,327220000000,0,327220000000,829946000000,410730000000
6,J P MORGAN CHASE & CO,Financial Services,387330000000,122900000000,0,122900000000,3386000000000,3106000000000
7,VISA,Financial Services,482170000000,21840000000,14090000000,21840000000,77884000000,42360000000
8,JOHNSON & JOHNSON,Healthcare,414310000000,82580000000,19730000000,82580000000,158380000000,95402000000
9,WALMART,Retail,407840000000,523960000000,22540000000,523960000000,237382000000,162072000000


In [46]:
string = "METLIFE Total Assets as on June-2020 780,30 is Million USD"
value = re.findall("\d+[.,]\d+|\b(?<!-)\d+\b", string)[0]

In [47]:
value.strip()

'780,30'

In [ ]:
"\d+[.,]\d+|\s\d+|^\d+"

In [ ]:
"\b(?<!-)\d+\b" - "negative lookbehind"